In [2]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')
!unzip 'drive/MyDrive/kubig_ML/235713_신용카드 사용자 연체 예측 AI 경진대회_data.zip'



Mounted at /content/drive
Archive:  drive/MyDrive/kubig_ML/235713_신용카드 사용자 연체 예측 AI 경진대회_data.zip
   creating: open/
  inflating: open/train.csv          
  inflating: open/sample_submission.csv  
  inflating: open/test.csv           


In [13]:
train=pd.read_csv('open/train.csv')
test=pd.read_csv('open/test.csv')

# 전처리 해본 것

In [14]:
#완전히 중복되는 데이터 삭제
train=train[~train.drop(columns=['credit','index','FLAG_MOBIL']).duplicated(keep='first')]

# day_employed 양수값 0으로 변환
train['DAYS_EMPLOYED'].loc[train['DAYS_EMPLOYED']>0]=0
test['DAYS_EMPLOYED'].loc[test['DAYS_EMPLOYED']>0]=0
train['occyp_type'].loc[(train['DAYS_EMPLOYED']==0)&(train['occyp_type'].isna())]='No job'
train['occyp_type']=train['occyp_type'].fillna('Laborers')
test['occyp_type'].loc[(test['DAYS_EMPLOYED']==0)&(test['occyp_type'].isna())]='No job'
test['occyp_type']=test['occyp_type'].fillna('Laborers')


# 성별 변수 인코딩
train['gender']=train['gender'].replace(['M','F'],[1,0])
train['car']=train['car'].replace(['Y','N'],[1,0])
train['reality']=train['reality'].replace(['Y','N'],[1,0])

train.drop(['index','FLAG_MOBIL'],axis=1,inplace=True)

# 자녀유무 변수
# train['kids']=0
# train['kids'] = np.where(train['child_num'] !=0, 1, 0)
# train.drop(['child_num'],axis=1,inplace=True)


# 인코딩 > ordinalencoder로 바꿈
from sklearn.preprocessing import LabelEncoder
#cat=train.dtypes[train.dtypes=="object"].index.tolist()
encoder = LabelEncoder()
# train[cat] = encoder.fit_transform(train[cat],train['credit'])
train['income_type'] = encoder.fit_transform(train['income_type'])
test['income_type']=encoder.transform(test['income_type'])
train['family_type'] = encoder.fit_transform(train['family_type'])
test['family_type']=encoder.transform(test['family_type'])
train['edu_type'] = encoder.fit_transform(train['edu_type'])
test['edu_type']=encoder.transform(test['edu_type'])
train['house_type'] = encoder.fit_transform(train['house_type'])
test['house_type']=encoder.transform(test['house_type'])
train['occyp_type'] = encoder.fit_transform(train['occyp_type'])
test['occyp_type']=encoder.transform(test['occyp_type'])

# family_size outlier 삭제
train=train[(train['family_size']<15)]
#train=train.reset_index(drop=True)

train['income_total']=np.log1p(train['income_total'])
# income_total, family_size 스케일링
# from sklearn.preprocessing import RobustScaler
# scaler3 = RobustScaler()
# scale=['income_total','family_size']
# train[scale] = scaler3.fit_transform(train[scale])
# test[scale] = scaler3.transform(test[scale])


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-14-6feb28d8283f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['occyp_type']=train['occyp_type'].fillna('Laborers')
<ipython-input-14-6feb28d8283f>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

# Test 제출용

## test 전처리

In [15]:


# 'gender','car','reality' 0,1로 encoding
test['gender']=test['gender'].replace(['M','F'],[1,0])
test['car']=test['car'].replace(['Y','N'],[1,0])
test['reality']=test['reality'].replace(['Y','N'],[1,0])

index=test['index'] # 나중에 index 쓸 것이라서 저장

#'FLAG_MOBIL','index' drop
test.drop(['index','FLAG_MOBIL'],axis=1,inplace=True)

# 자식 유무 변수 'kids' 추가 후 'child_num' drop
# test['kids']=0
# test['kids'] = np.where(test['child_num'] !=0, 1, 0)
# test.drop(['child_num'],axis=1,inplace=True)

test['income_total']=np.log1p(test['income_total'])



In [16]:
# train data X,y 만들어주기
X_train=train.drop(columns='credit')
y_train=train['credit']

In [17]:
from lightgbm import LGBMClassifier
model_lgbc = LGBMClassifier(n_estimators=450, max_depth=10,
                            colsample_bytree=1, min_child_weight=3, subsample=0.001, 
                            random_state=2)
#model_lgbc=LGBMClassifier()
model_lgbc.fit(X_train, y_train)

y_pred=model_lgbc.predict_proba(test)

In [18]:
y_pred

array([[0.05901116, 0.12468991, 0.81629893],
       [0.1814972 , 0.14985057, 0.66865222],
       [0.04793127, 0.07975212, 0.87231661],
       ...,
       [0.03866556, 0.10307931, 0.85825513],
       [0.14930303, 0.18979863, 0.66089833],
       [0.03469967, 0.22201035, 0.74328998]])

In [19]:
y_pred_df=pd.DataFrame(y_pred)

In [20]:
y_pred_df=pd.concat([index,y_pred_df],axis=1)
y_pred_df.to_csv('final2_submission.csv',index = False)